In [15]:
from pyspark.sql import SparkSession


spark = SparkSession.builder \
    .appName("PostgresConnectionTest") \
    .config("spark.jars", "/opt/spark/jars/postgresql-42.7.3.jar") \
    .getOrCreate()


jdbc_url = "jdbc:postgresql://postgres_general:5432/postgres"
jdbc_props = {
    "user": "admin",
    "password": "admin",
    "driver": "org.postgresql.Driver"
}


df = spark.createDataFrame(
    [(1, "Spark test connection"), (2, "Postgres test passed!")],
    ["id", "message"]
)

df.write \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "spark_test_table") \
    .options(**jdbc_props) \
    .mode("overwrite") \
    .save()

print("Successfully wrote test table to PostgreSQL!")


Successfully wrote test table to PostgreSQL!


In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SQLServerToPostgresDocker") \
    .config(
        "spark.jars",
        "/opt/spark/jars/mssql-jdbc-13.2.1.jre8.jar,/opt/spark/jars/postgresql-42.7.3.jar"
    ) \
    .getOrCreate()

sqlserver_url = "jdbc:sqlserver://192.168.1.7:1433;databaseName=EducationSystem_Raw;encrypt=false;trustServerCertificate=true;"

sqlserver_props = {
    "user": "sa",
    "password": "sa12345",
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

df = spark.read.jdbc(
    url=sqlserver_url,
    table="(SELECT TOP 10 * FROM dbo.Students) AS t",  # قراءة عينة صغيرة للتأكد
    properties=sqlserver_props
)

df.show(truncate=False)


+----------+---------+------------+------+----------+--------+--------+--------------+--------------+--------------+
|student_id|family_id|student_name|gender|birth_date|grade   |city    |email         |phone         |full_name     |
+----------+---------+------------+------+----------+--------+--------+--------------+--------------+--------------+
|1         |219      |Sara        |Male  |2015-06-06|Grade 7 |Tanta   |aya1@mail.com |017.82878e+007|Sara Khaled   |
|2         |244      |Heba        |Male  |2025-08-26|Grade 7 |Mansoura|ahm2@mail.com |018.16901e+007|Heba Nasser   |
|3         |312      |Hussein     |Female|2012-07-23|Grade 8 |Tanta   |nou3@mail.com |NULL          |Hussein Omar  |
|4         |4        |Mahmoud     |Female|2014-11-13|Grade 11|NULL    |ali4@mail.com |017.88741e+007|Mahmoud Nasser|
|5         |115      |NULL        |Male  |2022-12-11|Grade 9 |Tanta   |NULL          |019.33359e+007|NULL          |
|6         |75       |Omar        |Female|2018-04-25|Grade 7 |Ta

In [2]:
df_students = spark.read.jdbc(
    url=sqlserver_url,
    table="(SELECT * FROM dbo.Students) AS t",
    properties=sqlserver_props
)

print("📋 البيانات الأصلية:")
df_students.show( truncate=False)

📋 البيانات الأصلية:
+----------+---------+------------+------+----------+--------+----------+--------------+--------------+---------------+
|student_id|family_id|student_name|gender|birth_date|grade   |city      |email         |phone         |full_name      |
+----------+---------+------------+------+----------+--------+----------+--------------+--------------+---------------+
|1         |219      |Sara        |Male  |2015-06-06|Grade 7 |Tanta     |aya1@mail.com |017.82878e+007|Sara Khaled    |
|2         |244      |Heba        |Male  |2025-08-26|Grade 7 |Mansoura  |ahm2@mail.com |018.16901e+007|Heba Nasser    |
|3         |312      |Hussein     |Female|2012-07-23|Grade 8 |Tanta     |nou3@mail.com |NULL          |Hussein Omar   |
|4         |4        |Mahmoud     |Female|2014-11-13|Grade 11|NULL      |ali4@mail.com |017.88741e+007|Mahmoud Nasser |
|5         |115      |NULL        |Male  |2022-12-11|Grade 9 |Tanta     |NULL          |019.33359e+007|NULL           |
|6         |75      

In [3]:
students_clean = df_students.fillna({
    "student_name": "Unknown",
    "email": "unknown@gmail.com",
    "phone": "01000000000",
    "city": "Cairo",
    "grade": "Grade 10"
})
students_clean.show(truncate=False)

+----------+---------+------------+------+----------+--------+----------+-----------------+--------------+---------------+
|student_id|family_id|student_name|gender|birth_date|grade   |city      |email            |phone         |full_name      |
+----------+---------+------------+------+----------+--------+----------+-----------------+--------------+---------------+
|1         |219      |Sara        |Male  |2015-06-06|Grade 7 |Tanta     |aya1@mail.com    |017.82878e+007|Sara Khaled    |
|2         |244      |Heba        |Male  |2025-08-26|Grade 7 |Mansoura  |ahm2@mail.com    |018.16901e+007|Heba Nasser    |
|3         |312      |Hussein     |Female|2012-07-23|Grade 8 |Tanta     |nou3@mail.com    |01000000000   |Hussein Omar   |
|4         |4        |Mahmoud     |Female|2014-11-13|Grade 11|Cairo     |ali4@mail.com    |017.88741e+007|Mahmoud Nasser |
|5         |115      |Unknown     |Male  |2022-12-11|Grade 9 |Tanta     |unknown@gmail.com|019.33359e+007|NULL           |
|6         |75  

In [4]:
students_clean = students_clean.dropna(subset=["birth_date","family_id"])
students_clean = students_clean.dropDuplicates(["student_name", "email","family_id"])

students_clean.select("student_id", "student_name","email","family_id").show(20, truncate=False)

+----------+------------+-----------------+---------+
|student_id|student_name|email            |family_id|
+----------+------------+-----------------+---------+
|172       |Hussein     |mos164@mail.com  |231      |
|762       |Reem        |sar740@mail.com  |70       |
|954       |Nader       |oma926@mail.com  |33       |
|1034      |Unknown     |unknown@gmail.com|3        |
|1051      |Ali         |you1021@mail.com |152      |
|1249      |Aya         |unknown@gmail.com|71       |
|1465      |Zeinab      |lai1427@mail.com |321      |
|8         |Ahmed       |fat8@mail.com    |169      |
|521       |Hussein     |ali506@mail.com  |255      |
|679       |Nader       |tar658@mail.com  |69       |
|1107      |Nada        |unknown@gmail.com|122      |
|1443      |Ahmed       |kar1405@mail.com |195      |
|1469      |Unknown     |ree1431@mail.com |207      |
|116       |Fatma       |ree111@mail.com  |297      |
|157       |Amr         |unknown@gmail.com|160      |
|317       |Nour        |mah

In [17]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, current_date 
students_clean = students_clean.withColumn("effective_date", current_date()) \
                   .withColumn("end_date", lit("9999-12-31")) \
                   .withColumn("current_flag", lit(True))

In [40]:
courses_df = spark.read.jdbc(
    url=sqlserver_url,
    table="(SELECT * FROM dbo.courses) AS t",
    properties=sqlserver_props
)

print("📋 البيانات الأصلية:")
courses_df.show( truncate=False)

📋 البيانات الأصلية:
+---------+----------------+-----------------+--------+
|course_id|course_name     |teacher_name     |semester|
+---------+----------------+-----------------+--------+
|1        |Mathematics     |NULL             |1       |
|2        |Physics         |NULL             |1       |
|3        |Chemistry       |NULL             |1       |
|4        |English         |Dr. Mona Hassan  |1       |
|5        |Arabic          |Dr. Heba Youssef |1       |
|6        |Biology         |Dr. Karim Fathy  |2       |
|7        |History         |Dr. Yara Mostafa |2       |
|8        |Geography       |Dr. Adel Samir   |2       |
|9        |Computer Science|Dr. Hossam Tarek |2       |
|10       |Philosophy      |Dr. Rana Ahmed   |2       |
|11       |French          |Dr. Sarah Ibrahim|1       |
|12       |Religion        |Dr. Tarek Mahmoud|1       |
|13       |Economics       |Dr. Waleed Fathy |2       |
|14       |Sociology       |Dr. Heba Magdy   |2       |
|15       |Statistics      |

In [41]:
courses_clean = courses_df.fillna({
    "teacher_name": "Unknown Teacher",
    "semester": "1"
})
courses_clean.show(truncate=False)

+---------+----------------+-----------------+--------+
|course_id|course_name     |teacher_name     |semester|
+---------+----------------+-----------------+--------+
|1        |Mathematics     |Unknown Teacher  |1       |
|2        |Physics         |Unknown Teacher  |1       |
|3        |Chemistry       |Unknown Teacher  |1       |
|4        |English         |Dr. Mona Hassan  |1       |
|5        |Arabic          |Dr. Heba Youssef |1       |
|6        |Biology         |Dr. Karim Fathy  |2       |
|7        |History         |Dr. Yara Mostafa |2       |
|8        |Geography       |Dr. Adel Samir   |2       |
|9        |Computer Science|Dr. Hossam Tarek |2       |
|10       |Philosophy      |Dr. Rana Ahmed   |2       |
|11       |French          |Dr. Sarah Ibrahim|1       |
|12       |Religion        |Dr. Tarek Mahmoud|1       |
|13       |Economics       |Dr. Waleed Fathy |2       |
|14       |Sociology       |Dr. Heba Magdy   |2       |
|15       |Statistics      |Dr. Nader Zaki   |2 

In [42]:
course_student_df = students_clean.crossJoin(courses_clean) \
    .withColumn("enrollment_date", current_date()) \
    .withColumn("completion_date", lit(None)) \
    .withColumn("grade", lit(None))

In [43]:
enroll_df = spark.read.jdbc(
    url=sqlserver_url,
    table="(SELECT * FROM dbo.Enrollments) AS t",
    properties=sqlserver_props
)

print("📋 البيانات الأصلية:")
enroll_df.show( truncate=False)

📋 البيانات الأصلية:
+-------------+----------+---------+---------------+
|enrollment_id|student_id|course_id|enrollment_date|
+-------------+----------+---------+---------------+
|1            |965       |1        |2024-05-27     |
|2            |1123      |2        |2024-10-02     |
|3            |1354      |8        |2024-03-30     |
|4            |173       |15       |2025-02-15     |
|5            |1287      |NULL     |2025-06-13     |
|6            |1385      |5        |2025-09-02     |
|7            |1153      |10       |2024-05-12     |
|8            |184       |4        |2023-08-01     |
|9            |NULL      |9        |2025-08-01     |
|10           |461       |1        |2025-08-07     |
|11           |492       |5        |2025-04-23     |
|12           |232       |14       |2023-07-14     |
|13           |19        |13       |2025-01-08     |
|14           |973       |9        |2023-08-07     |
|15           |948       |12       |2025-03-28     |
|16           |1108      |

In [44]:
enroll_clean = enroll_df.dropDuplicates(["student_id", "course_id"])
enroll_clean = enroll_clean.dropna(subset=["student_id", "course_id"])
enroll_clean.show(truncate=False)

+-------------+----------+---------+---------------+
|enrollment_id|student_id|course_id|enrollment_date|
+-------------+----------+---------+---------------+
|180          |79        |10       |2024-04-26     |
|250          |83        |11       |2024-12-04     |
|385          |1010      |4        |2025-07-18     |
|387          |964       |14       |2023-10-25     |
|605          |309       |9        |2024-01-15     |
|733          |1023      |15       |2024-05-01     |
|735          |1326      |6        |2023-06-14     |
|862          |276       |11       |2025-09-22     |
|1397         |1307      |4        |2023-06-28     |
|1830         |336       |5        |2023-08-12     |
|1974         |794       |13       |2024-09-10     |
|2021         |426       |13       |2024-10-10     |
|2066         |973       |10       |2023-04-26     |
|2458         |114       |15       |2025-05-20     |
|2517         |636       |8        |2024-12-05     |
|2553         |1408      |6        |2024-07-05

In [18]:
family_df = spark.read.jdbc(
    url=sqlserver_url,
    table="(SELECT * FROM dbo.Family) AS t",
    properties=sqlserver_props
)

print("📋 البيانات الأصلية:")
family_df.show( truncate=False)

📋 البيانات الأصلية:
+---------+--------------+------------+---------------+----------+-------------------------+
|family_id|father_name   |mother_name |contact_number |address   |email                    |
+---------+--------------+------------+---------------+----------+-------------------------+
|701      |Tarek         |NULL        |0109.69622e+007|Suez      |tarek.1@gmail.com        |
|702      |Ahmed         |NULL        |0102.5946e+007 |Zagazig   |ahmed.2@gmail.com        |
|703      |Hassan Ibrahim|Noura Samira|0109.77152e+007|Aswan     |hassan.ibrahim3@gmail.com|
|704      |Youssef       |Abeer       |0109.01294e+006|Giza      |youssef.4@gmail.com      |
|705      | Zaki         |Hoda Hassan |0107.29133e+007|Alexandria|.zaki5@gmail.com         |
|706      |Ahmed         |Sara Tarek  |NULL           |Suez      |ahmed.6@gmail.com        |
|707      | Nabil        |            |0106.338e+007  |Tanta     |.nabil7@gmail.com        |
|708      |Omar Fathy    |Walaa       |0107.78859e

In [19]:

family_clean_ = family_df.replace(' ', None)
family_clean_=family_clean_.dropna(subset=["family_id"])

family_clean_.show(truncate=False)

+---------+--------------+------------+---------------+----------+-------------------------+
|family_id|father_name   |mother_name |contact_number |address   |email                    |
+---------+--------------+------------+---------------+----------+-------------------------+
|701      |Tarek         |NULL        |0109.69622e+007|Suez      |tarek.1@gmail.com        |
|702      |Ahmed         |NULL        |0102.5946e+007 |Zagazig   |ahmed.2@gmail.com        |
|703      |Hassan Ibrahim|Noura Samira|0109.77152e+007|Aswan     |hassan.ibrahim3@gmail.com|
|704      |Youssef       |Abeer       |0109.01294e+006|Giza      |youssef.4@gmail.com      |
|705      | Zaki         |Hoda Hassan |0107.29133e+007|Alexandria|.zaki5@gmail.com         |
|706      |Ahmed         |Sara Tarek  |NULL           |Suez      |ahmed.6@gmail.com        |
|707      | Nabil        |NULL        |0106.338e+007  |Tanta     |.nabil7@gmail.com        |
|708      |Omar Fathy    |Walaa       |0107.78859e+007|Minya     |omar

In [20]:
family_clean_ = family_clean_.fillna({
    "address": "Unknown",
    "email": "unknown@gmail.com",
    "contact_number": "01000000000",
    "father_name" : "Unknown",
    "mother_name" : "Unknown",
})
family_clean_.show(truncate=False)

+---------+--------------+------------+---------------+----------+-------------------------+
|family_id|father_name   |mother_name |contact_number |address   |email                    |
+---------+--------------+------------+---------------+----------+-------------------------+
|701      |Tarek         |Unknown     |0109.69622e+007|Suez      |tarek.1@gmail.com        |
|702      |Ahmed         |Unknown     |0102.5946e+007 |Zagazig   |ahmed.2@gmail.com        |
|703      |Hassan Ibrahim|Noura Samira|0109.77152e+007|Aswan     |hassan.ibrahim3@gmail.com|
|704      |Youssef       |Abeer       |0109.01294e+006|Giza      |youssef.4@gmail.com      |
|705      | Zaki         |Hoda Hassan |0107.29133e+007|Alexandria|.zaki5@gmail.com         |
|706      |Ahmed         |Sara Tarek  |01000000000    |Suez      |ahmed.6@gmail.com        |
|707      | Nabil        |Unknown     |0106.338e+007  |Tanta     |.nabil7@gmail.com        |
|708      |Omar Fathy    |Walaa       |0107.78859e+007|Minya     |omar

In [21]:
family_clean=family_clean_
family_clean.show()

+---------+--------------+------------+---------------+----------+--------------------+
|family_id|   father_name| mother_name| contact_number|   address|               email|
+---------+--------------+------------+---------------+----------+--------------------+
|      701|        Tarek |     Unknown|0109.69622e+007|      Suez|   tarek.1@gmail.com|
|      702|        Ahmed |     Unknown| 0102.5946e+007|   Zagazig|   ahmed.2@gmail.com|
|      703|Hassan Ibrahim|Noura Samira|0109.77152e+007|     Aswan|hassan.ibrahim3@g...|
|      704|      Youssef |      Abeer |0109.01294e+006|      Giza| youssef.4@gmail.com|
|      705|          Zaki| Hoda Hassan|0107.29133e+007|Alexandria|    .zaki5@gmail.com|
|      706|        Ahmed |  Sara Tarek|    01000000000|      Suez|   ahmed.6@gmail.com|
|      707|         Nabil|     Unknown|  0106.338e+007|     Tanta|   .nabil7@gmail.com|
|      708|    Omar Fathy|      Walaa |0107.78859e+007|     Minya|omar.fathy8@gmail...|
|      709|    Khaled Ali|     U

In [22]:
from pyspark.sql.functions import lit

family_clean = family_clean.withColumn("family_email", lit("unknown@gmail.com")) \
                           .withColumn("family_address", lit("Unknown"))



In [23]:
family_clean = family_clean.drop("family_email", "family_address")


In [24]:
from pyspark.sql.functions import lit

family_clean = family_clean.fillna({
    "father_name": "Unknown",
    "mother_name": "Unknown",
    "contact_number": "0000000000",
    "address": "Unknown",
    "email": "N/A"
})


In [25]:
from pyspark.sql.functions import monotonically_increasing_id

family_clean = family_clean.withColumn("family_id", 
                                                 lit(None).cast("int")) \
                                      .withColumn("family_id", 
                                                  monotonically_increasing_id())


In [26]:
family_clean = family_clean.drop("family_email", "family_address")


In [27]:
from pyspark.sql.functions import col

family_clean = family_clean.withColumn("family_id", col("family_id").cast("string"))


In [80]:
family_clean = family_clean.fillna({
    "father_name": "Unknown",
    "mother_name": "Unknown",
    "email": "N/A",
    "contact_number": "N/A",
    "address": "N/A"
})


In [48]:
teacher_clean = spark.read.jdbc(
    url=sqlserver_url,
    table="(SELECT * FROM dbo.DimTeacher) AS t",
    properties=sqlserver_props
)

print("📋 البيانات الأصلية:")
teacher_clean.show( truncate=False)

📋 البيانات الأصلية:
+----------+------------+-----------+----------+----------+
|teacher_id|teacher_name|phone      |hire_date |department|
+----------+------------+-----------+----------+----------+
|T001      |Ahmed Ali   |01012345678|2020-09-01|Math      |
|T002      |Sara Mohamed|01087654321|2019-01-15|Physics   |
|T003      |Omar Hassan |01011223344|2021-06-20|Chemistry |
+----------+------------+-----------+----------+----------+



In [49]:
exams_df = spark.read.jdbc(
    url=sqlserver_url,
    table="(SELECT * FROM dbo.Exams) AS t",
    properties=sqlserver_props
)

print("📋 البيانات الأصلية:")
exams_df.show( truncate=False)

📋 البيانات الأصلية:
+-------+----------+---------+-----+----------+
|exam_id|student_id|course_id|score|exam_date |
+-------+----------+---------+-----+----------+
|1      |1453      |15       |12.0 |2022-12-11|
|2      |878       |10       |77.65|2025-08-25|
|3      |1111      |2        |64.6 |2024-05-26|
|4      |866       |5        |43.07|2022-10-12|
|5      |1272      |NULL     |81.59|2025-07-03|
|6      |1171      |1        |37.35|2022-12-06|
|7      |976       |6        |72.99|2024-03-09|
|8      |NULL      |10       |56.66|2025-05-27|
|9      |1432      |11       |76.35|2022-08-20|
|10     |960       |1        |88.61|2024-11-05|
|11     |1472      |13       |51.17|2024-04-06|
|12     |965       |9        |64.23|2025-06-24|
|13     |298       |10       |67.7 |2022-09-23|
|14     |122       |NULL     |NULL |2022-07-27|
|15     |NULL      |5        |20.62|2023-09-16|
|16     |867       |2        |NULL |2023-04-22|
|17     |527       |3        |NULL |2023-05-19|
|18     |1046      |

In [50]:
exams_clean=exams_df.dropna(subset=["student_id"])
exams_clean=exams_df.dropna(subset=["course_id"])

exams_clean.show(truncate=False)


+-------+----------+---------+-----+----------+
|exam_id|student_id|course_id|score|exam_date |
+-------+----------+---------+-----+----------+
|1      |1453      |15       |12.0 |2022-12-11|
|2      |878       |10       |77.65|2025-08-25|
|3      |1111      |2        |64.6 |2024-05-26|
|4      |866       |5        |43.07|2022-10-12|
|6      |1171      |1        |37.35|2022-12-06|
|7      |976       |6        |72.99|2024-03-09|
|8      |NULL      |10       |56.66|2025-05-27|
|9      |1432      |11       |76.35|2022-08-20|
|10     |960       |1        |88.61|2024-11-05|
|11     |1472      |13       |51.17|2024-04-06|
|12     |965       |9        |64.23|2025-06-24|
|13     |298       |10       |67.7 |2022-09-23|
|15     |NULL      |5        |20.62|2023-09-16|
|16     |867       |2        |NULL |2023-04-22|
|17     |527       |3        |NULL |2023-05-19|
|18     |1046      |8        |2.48 |2023-02-24|
|19     |702       |3        |73.71|2023-09-02|
|20     |146       |7        |46.04|2023

In [51]:
from pyspark.sql import functions as Fn 
avg_score = exams_df.select(Fn.avg(Fn.col("score"))).collect()[0][0]
exams_clean = exams_clean.fillna({"score": avg_score})

In [52]:
exams_clean.show(truncate=False)

+-------+----------+---------+------------------+----------+
|exam_id|student_id|course_id|score             |exam_date |
+-------+----------+---------+------------------+----------+
|1      |1453      |15       |12.0              |2022-12-11|
|2      |878       |10       |77.65             |2025-08-25|
|3      |1111      |2        |64.6              |2024-05-26|
|4      |866       |5        |43.07             |2022-10-12|
|6      |1171      |1        |37.35             |2022-12-06|
|7      |976       |6        |72.99             |2024-03-09|
|8      |NULL      |10       |56.66             |2025-05-27|
|9      |1432      |11       |76.35             |2022-08-20|
|10     |960       |1        |88.61             |2024-11-05|
|11     |1472      |13       |51.17             |2024-04-06|
|12     |965       |9        |64.23             |2025-06-24|
|13     |298       |10       |67.7              |2022-09-23|
|15     |NULL      |5        |20.62             |2023-09-16|
|16     |867       |2   

In [53]:
# عمل العمود result_flag
exams_clean = exams_clean.withColumn(
    "result_flag",
    when(col("score") >= 50, True).otherwise(False)
)

In [54]:
attendance_df = spark.read.jdbc(
    url=sqlserver_url,
    table="(SELECT * FROM dbo.Attendance) AS t",
    properties=sqlserver_props
)

print("📋 البيانات الأصلية:")
attendance_df.show( truncate=False)

📋 البيانات الأصلية:
+-------------+----------+---------+---------------+-------+
|attendance_id|student_id|course_id|attendance_date|status |
+-------------+----------+---------+---------------+-------+
|1            |174       |10       |2024-10-15     |Present|
|2            |1134      |3        |2024-01-02     |Absent |
|3            |NULL      |8        |2024-07-19     |Present|
|4            |951       |1        |2025-10-22     |Present|
|5            |715       |12       |2025-02-13     |Present|
|6            |329       |13       |2025-05-01     |Present|
|7            |1226      |3        |2025-01-06     |Present|
|8            |36        |NULL     |2025-08-01     |Present|
|9            |777       |5        |2024-03-24     |Absent |
|10           |775       |4        |2025-01-08     |Absent |
|11           |985       |8        |2024-04-16     |Absent |
|12           |1254      |14       |2024-05-08     |Absent |
|13           |1292      |12       |2024-12-11     |Present|
|14 

In [55]:
attendance_clean=attendance_df.dropna(subset=["student_id"])
attendance_clean=attendance_df.dropna(subset=["course_id"])
attendance_clean = attendance_clean.fillna({"status": "unknown"})

attendance_clean.show(truncate=False)

+-------------+----------+---------+---------------+-------+
|attendance_id|student_id|course_id|attendance_date|status |
+-------------+----------+---------+---------------+-------+
|1            |174       |10       |2024-10-15     |Present|
|2            |1134      |3        |2024-01-02     |Absent |
|3            |NULL      |8        |2024-07-19     |Present|
|4            |951       |1        |2025-10-22     |Present|
|5            |715       |12       |2025-02-13     |Present|
|6            |329       |13       |2025-05-01     |Present|
|7            |1226      |3        |2025-01-06     |Present|
|9            |777       |5        |2024-03-24     |Absent |
|10           |775       |4        |2025-01-08     |Absent |
|11           |985       |8        |2024-04-16     |Absent |
|12           |1254      |14       |2024-05-08     |Absent |
|13           |1292      |12       |2024-12-11     |Present|
|14           |907       |1        |2024-06-23     |Present|
|15           |NULL     

In [ ]:
students_clean.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres_general:5432/dwh") \
    .option("dbtable", "dimstsudents") \
    .options(user="admin", password="admin", driver="org.postgresql.Driver") \
    .mode("overwrite") \
    .save()

print("✅ Table Dimstudents uploaded successfully!")


In [ ]:
family_clean.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres_general:5432/dwh") \
    .option("dbtable", "dimfamily") \
    .options(user="admin", password="admin", driver="org.postgresql.Driver") \
    .mode("overwrite") \
    .save()

print("✅ Table DimFamily uploaded successfully!")


In [112]:
    
 family_clean.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres_general:5432/dwh") \
    .option("dbtable", "dimfamily") \
    .options(user="admin", password="admin", driver="org.postgresql.Driver") \
    .mode("overwrite") \
    .save()

print("✅ Table DimFamily uploaded successfully!")


✅ Table DimFamily uploaded successfully!


In [110]:
 DimTeacher.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres_general:5432/dwh") \
    .option("dbtable", "DimTeacher") \
    .options(user="admin", password="admin", driver="org.postgresql.Driver") \
    .mode("overwrite") \
    .save()

print("✅ Table DimTeacher uploaded successfully!")


✅ Table DimTeacher uploaded successfully!


+----------+------------+-----------+----------+----------+
|teacher_id|teacher_name|      phone| hire_date|department|
+----------+------------+-----------+----------+----------+
|      T001|   Ahmed Ali|01012345678|2020-09-01|      Math|
|      T002|Sara Mohamed|01087654321|2019-01-15|   Physics|
|      T003| Omar Hassan|01011223344|2021-06-20| Chemistry|
+----------+------------+-----------+----------+----------+



In [104]:
from pyspark.sql.functions import lit, col

# افترض عندك DataFrame courses_df بالبيانات اللي بعتتهالي
# افترض عندك DataFrame dim_teacher_df فيه teacher_id و teacher_name

# أول حاجة نعمل join عشان نحصل على teacher_id
courses_df_ready = courses_clean.join(
    DimTeacher.select("teacher_id", "teacher_name"),
    on="teacher_name",
    how="left"
).withColumn("effective_date", lit("2025-11-05")) \
 .select(
     col("course_id").cast("string"),
     col("course_name").cast("string"),
     col("semester").cast("string"),
     col("teacher_id").cast("string"),
     col("effective_date").cast("date")
 )

In [109]:
 courses_df_ready.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres_general:5432/dwh") \
    .option("dbtable", "courses_df_ready") \
    .options(user="admin", password="admin", driver="org.postgresql.Driver") \
    .mode("overwrite") \
    .save()

print("✅ Table courses_df_ready uploaded successfully!")

✅ Table courses_df_ready uploaded successfully!


In [59]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, explode, sequence
from pyspark.sql.types import DateType
import datetime

spark = SparkSession.builder.getOrCreate()

# توليد تواريخ بين 2025-01-01 و 2025-12-31
start = datetime.date(2025, 1, 1)
end = datetime.date(2025, 12, 31)

date_df = spark.range(0, (end - start).days + 1) \
    .withColumn("full_date", (lit(start) + col("id").cast("interval day"))) \
    .withColumn("date_id", col("full_date")) \
    .withColumn("day", col("full_date").cast("date").substr(9, 2).cast("int")) \
    .withColumn("month", col("full_date").cast("date").substr(6, 2).cast("int")) \
    .withColumn("year", col("full_date").cast("date").substr(1, 4).cast("int")) \
    .drop("id")

In [114]:
from pyspark.sql.functions import lit

# افترض عندك DataFrame اسمه enrollments_df فيه student_id و course_id
# لو مش عندك تاريخ، ممكن تحط التاريخ الحالي مؤقتاً
enroll_clean = enroll_clean.withColumn("date_id", lit("2025-11-05")) \
                                     .withColumn("enrollment_count", lit(1))

# رفع على PostgreSQL
enroll_clean.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres_general:5432/dwh") \
    .option("dbtable", "fact_enroll") \
    .options(user="admin", password="admin", driver="org.postgresql.Driver") \
    .mode("overwrite") \
    .save()

print("✅ Table enroll_clean uploaded successfully!")


✅ Table enroll_clean uploaded successfully!


In [115]:
from pyspark.sql.functions import lit

# افترض عندك DataFrame اسمه attendance_df فيه student_id و course_id
# نضيف تاريخ افتراضي للحضور وحالة الحضور
attendance_clean = attendance_clean.withColumn("date_id", lit("2025-11-05")) \
                                   .withColumn("status", lit(True))  # True يعني حضر

# رفع على PostgreSQL
attendance_clean.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres_general:5432/dwh") \
    .option("dbtable", "FactAttendance") \
    .options(user="admin", password="admin", driver="org.postgresql.Driver") \
    .mode("overwrite") \
    .save()

print("✅ Table attendance_clean uploaded successfully!")


✅ Table attendance_clean uploaded successfully!


In [116]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, explode, sequence
from pyspark.sql.types import DateType
import datetime

spark = SparkSession.builder.getOrCreate()

# توليد تواريخ بين 2025-01-01 و 2025-12-31
start = datetime.date(2025, 1, 1)
end = datetime.date(2025, 12, 31)

date_df = spark.range(0, (end - start).days + 1) \
    .withColumn("full_date", (lit(start) + col("id").cast("interval day"))) \
    .withColumn("date_id", col("full_date")) \
    .withColumn("day", col("full_date").cast("date").substr(9, 2).cast("int")) \
    .withColumn("month", col("full_date").cast("date").substr(6, 2).cast("int")) \
    .withColumn("year", col("full_date").cast("date").substr(1, 4).cast("int")) \
    .drop("id")

# رفع على PostgreSQL
date_df.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres_general:5432/dwh") \
    .option("dbtable", "DimDate") \
    .options(user="admin", password="admin", driver="org.postgresql.Driver") \
    .mode("overwrite") \
    .save()

print("✅ Table DimDate uploaded successfully!")


✅ Table DimDate uploaded successfully!


In [ ]:
exams_clean.show()


+-------+----------+---------+------------------+----------+
|exam_id|student_id|course_id|             score| exam_date|
+-------+----------+---------+------------------+----------+
|      1|      1453|       15|              12.0|2022-12-11|
|      2|       878|       10|             77.65|2025-08-25|
|      3|      1111|        2|              64.6|2024-05-26|
|      4|       866|        5|             43.07|2022-10-12|
|      6|      1171|        1|             37.35|2022-12-06|
|      7|       976|        6|             72.99|2024-03-09|
|      8|      NULL|       10|             56.66|2025-05-27|
|      9|      1432|       11|             76.35|2022-08-20|
|     10|       960|        1|             88.61|2024-11-05|
|     11|      1472|       13|             51.17|2024-04-06|
|     12|       965|        9|             64.23|2025-06-24|
|     13|       298|       10|              67.7|2022-09-23|
|     15|      NULL|        5|             20.62|2023-09-16|
|     16|       867|    

In [121]:
  exams_clean.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres_general:5432/dwh") \
    .option("dbtable", "fact_exam") \
    .options(user="admin", password="admin", driver="org.postgresql.Driver") \
    .mode("overwrite") \
    .save()


print("✅ Table exams_clean uploaded successfully!")

✅ Table exams_clean uploaded successfully!


In [134]:
if "course_student_id" in course_student_df.columns:
    course_student_df = course_student_df.drop("course_student_id")

In [136]:
from pyspark.sql.types import StringType

course_student_df = course_student_df.withColumn("grade", course_student_df["grade"].cast(StringType()))


In [138]:
from pyspark.sql.types import StringType, DateType

course_student_df = course_student_df.withColumn("grade", course_student_df["grade"].cast(StringType()))
course_student_df = course_student_df.withColumn("enrollment_date", course_student_df["enrollment_date"].cast(DateType()))
course_student_df = course_student_df.withColumn("completion_date", course_student_df["completion_date"].cast(DateType()))


In [139]:
# لو فيه عمود dummy أو placeholder
for col_name, dtype in course_student_df.dtypes:
    if dtype == "null":
        course_student_df = course_student_df.drop(col_name)


In [140]:
course_student_df.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres_general:5432/dwh") \
    .option("dbtable", "DimCourseStudent") \
    .options(user="admin", password="admin", driver="org.postgresql.Driver") \
    .mode("overwrite") \
    .save()


In [28]:
family_clean.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres_general:5432/dwh") \
    .option("dbtable", "dimfamily") \
    .options(user="admin", password="admin", driver="org.postgresql.Driver") \
    .mode("overwrite") \
    .save()

print("✅ Table DimFamily uploaded successfully!")

Py4JJavaError: An error occurred while calling o177.save.
: org.postgresql.util.PSQLException: ERROR: cannot drop table dimfamily because other objects depend on it
  Detail: constraint fk_family on table dimstsudents depends on table dimfamily
  Hint: Use DROP ... CASCADE to drop the dependent objects too.
	at org.postgresql.core.v3.QueryExecutorImpl.receiveErrorResponse(QueryExecutorImpl.java:2725)
	at org.postgresql.core.v3.QueryExecutorImpl.processResults(QueryExecutorImpl.java:2412)
	at org.postgresql.core.v3.QueryExecutorImpl.execute(QueryExecutorImpl.java:371)
	at org.postgresql.jdbc.PgStatement.executeInternal(PgStatement.java:502)
	at org.postgresql.jdbc.PgStatement.execute(PgStatement.java:419)
	at org.postgresql.jdbc.PgStatement.executeWithFlags(PgStatement.java:341)
	at org.postgresql.jdbc.PgStatement.executeCachedSql(PgStatement.java:326)
	at org.postgresql.jdbc.PgStatement.executeWithFlags(PgStatement.java:302)
	at org.postgresql.jdbc.PgStatement.executeUpdate(PgStatement

In [29]:
family_clean.write\
    .format("jdbc")\
    .option("url", postgres_url)\
    .option("dbtable", "dimfamily")\
    .options(**postgres_props)\
    .mode("append")\
    .save()


In [31]:
students_clean.write \
    .format("jdbc")\
    .option("url", postgres_url)\
    .option("dbtable", "dimstsudents")\
    .options(**postgres_props)\
    .mode("append")\
    .save()

print("✅ Table Dimstudents uploaded successfully!")

✅ Table Dimstudents uploaded successfully!


In [60]:
date_df.printSchema()


root
 |-- full_date: date (nullable = false)
 |-- date_id: date (nullable = false)
 |-- day: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)



#break
##########################################################3

In [66]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

jdbc_url_postgres = "jdbc:postgresql://postgres_general:5432/dwh1"
jdbc_props_postgres = {
    "user": "admin",
    "password": "admin",
    "driver": "org.postgresql.Driver"
}

# test connection
spark.read.jdbc(url=jdbc_url_postgres, table="pg_tables", properties=jdbc_props_postgres).show(5)

+----------+----------+----------+----------+----------+--------+-----------+-----------+
|schemaname| tablename|tableowner|tablespace|hasindexes|hasrules|hastriggers|rowsecurity|
+----------+----------+----------+----------+----------+--------+-----------+-----------+
|    public| dimfamily|     admin|      NULL|      true|   false|       true|      false|
|    public|dimstudent|     admin|      NULL|      true|   false|       true|      false|
|    public|dimteacher|     admin|      NULL|      true|   false|       true|      false|
|    public| dimcourse|     admin|      NULL|      true|   false|       true|      false|
|    public|  factexam|     admin|      NULL|     false|   false|       true|      false|
+----------+----------+----------+----------+----------+--------+-----------+-----------+
only showing top 5 rows



In [72]:
jdbc_url_postgres = "jdbc:postgresql://postgres_general:5432/dwh1"

jdbc_props_postgres = {
    "user": "admin",
    "password": "admin",
    "driver": "org.postgresql.Driver"
}

# 1️⃣ Load Dimensions


students_clean.write.jdbc(url=jdbc_url_postgres, table="dimstudent", mode="overwrite", properties=jdbc_props_postgres)
teacher_clean.write.jdbc(url=jdbc_url_postgres, table="dimteacher", mode="overwrite", properties=jdbc_props_postgres)
course_student_df.write.jdbc(url=jdbc_url_postgres, table="dimcourse", mode="overwrite", properties=jdbc_props_postgres)
date_df.write.jdbc(url=jdbc_url_postgres, table="dimdate", mode="overwrite", properties=jdbc_props_postgres)

exams_clean.write.jdbc(url=jdbc_url_postgres, table="factexam", mode="overwrite", properties=jdbc_props_postgres)
attendance_clean.write.jdbc(url=jdbc_url_postgres, table="factattendance", mode="overwrite", properties=jdbc_props_postgres)
enroll_clean.write.jdbc(url=jdbc_url_postgres, table="factenrollment", mode="overwrite", properties=jdbc_props_postgres)

Py4JJavaError: An error occurred while calling o469.jdbc.
: org.postgresql.util.PSQLException: ERROR: cannot drop table dimstudent because other objects depend on it
  Detail: constraint factexam_student_id_fkey on table factexam depends on table dimstudent
constraint factattendance_student_id_fkey on table factattendance depends on table dimstudent
constraint dimcoursestudent_student_id_fkey on table dimcoursestudent depends on table dimstudent
  Hint: Use DROP ... CASCADE to drop the dependent objects too.
	at org.postgresql.core.v3.QueryExecutorImpl.receiveErrorResponse(QueryExecutorImpl.java:2725)
	at org.postgresql.core.v3.QueryExecutorImpl.processResults(QueryExecutorImpl.java:2412)
	at org.postgresql.core.v3.QueryExecutorImpl.execute(QueryExecutorImpl.java:371)
	at org.postgresql.jdbc.PgStatement.executeInternal(PgStatement.java:502)
	at org.postgresql.jdbc.PgStatement.execute(PgStatement.java:419)
	at org.postgresql.jdbc.PgStatement.executeWithFlags(PgStatement.java:341)
	at or

In [1]:
family_clean = family_clean.dropDuplicates(["email"])
students_clean = students_clean.dropDuplicates(["student_id"])
teacher_clean = teacher_clean.dropDuplicates(["teacher_id"])

NameError: name 'family_clean' is not defined

In [75]:
from pyspark.sql.functions import col
family_clean = family_clean.withColumn("family_id", col("family_id").cast("int"))
teacher_clean = teacher_clean.withColumn("teacher_id", col("teacher_id").cast("int"))

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 37922)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.11/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.11/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.11/socketserver.py", line 755, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 267, in poll
    if self.rfile in r and func():
                           ^^^^^^
  File "/usr/local/spark/python/pyspark/accumulators.py", line 271, in accum_updates
    num_updates =

In [ ]:
family_clean.write.jdbc(url=jdbc_url_postgres, table="dimfamily", mode="append", properties=jdbc_props_postgres)
students_clean.write.jdbc(url=jdbc_url_postgres, table="dimstudent", mode="append", properties=jdbc_props_postgres)
teacher_clean.write.jdbc(url=jdbc_url_postgres, table="dimteacher", mode="append", properties=jdbc_props_postgres)
enroll_Clean.write.jdbc(url=jdbc_url_postgres, table="factenrollment", mode="append", properties=jdbc_props_postgres)
course_student_df.write.jdbc(url=jdbc_url_postgres, table="factcoursestudent", mode="append", properties=jdbc_props_postgres)
exams_clean.write.jdbc(url=jdbc_url_postgres, table="factexam", mode="append", properties=jdbc_props_postgres)
attendance_clean.write.jdbc(url=jdbc_url_postgres, table="factattendance", mode="append", properties=jdbc_props_postgres)